<a href="https://colab.research.google.com/github/rjrizani/pdf_scraper/blob/main/pdf_to_csv_konstruksi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 62.7 MB/s eta 0:00:00


# Task
Extract 'NAMA PERUSAHAAN', 'KUALIFIKASI', and 'E-MAIL USAHA' from the PDF file "/content/kontruksi_2024_jateng.pdf" and save the extracted data into a CSV file. If the previous method failed, try an alternative approach to extract the data.

## Load pdf

### Subtask:
Load the PDF file using a library like `pdfplumber`.


**Reasoning**:
Import the `pdfplumber` library and open the PDF file.



In [15]:
import pdfplumber

pdf = pdfplumber.open("/content/kontruksi_2024_jateng.pdf")

## Extract text

### Subtask:
Extract text line by line from the PDF pages instead of extracting tables.


**Reasoning**:
Extract text line by line from each page of the PDF and store them in a list.



In [16]:
all_lines = []
for page in pdf.pages:
  text = page.extract_text()
  lines = text.splitlines()
  all_lines.extend(lines)

## Process data

### Subtask:
Iterate through the extracted text lines. Use regular expressions or pattern matching to identify and extract company names, qualifications, and email addresses from each relevant line.


**Reasoning**:
Initialize empty lists and iterate through the lines, using regular expressions to extract the required information.



In [17]:
import re

company_names = []
qualifications = []
emails = []

# Regex patterns to identify company names, qualifications, and emails
# This is a basic pattern and might need adjustment based on the actual data format
company_pattern = re.compile(r'^\d+\s+(.*)') # Starts with a number, followed by company name
qualification_pattern = re.compile(r'\b(BESAR|MENENGAH|KECIL)\b') # Looks for specific qualification keywords
email_pattern = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}') # Standard email pattern

for line in all_lines:
    line = line.strip() # Remove leading/trailing whitespace

    # Check if the line potentially contains company information
    if re.match(r'^\d+\s+', line): # Line starts with a number followed by a space
        company_match = company_pattern.match(line)
        if company_match:
            company_names.append(company_match.group(1).strip())
        else:
            company_names.append(None) # Append None if company name not found

        qualification_match = qualification_pattern.search(line)
        if qualification_match:
            qualifications.append(qualification_match.group(1))
        else:
            qualifications.append(None) # Append None if qualification not found

        email_match = email_pattern.search(line)
        if email_match:
            emails.append(email_match.group(0))
        else:
            emails.append(None) # Append None if email not found


**Reasoning**:
Create a DataFrame from the extracted lists and display the first few rows to verify the extraction.



In [18]:
import pandas as pd

df_extracted = pd.DataFrame({
    'NAMA PERUSAHAAN': company_names,
    'KUALIFIKASI': qualifications,
    'E-MAIL USAHA': emails
})

display(df_extracted.head())

,NAMA PERUSAHAAN,KUALIFIKASI,E-MAIL USAHA
0,Direktori Perusahaan Konstruksi Provinsi Jawa ...,None,None
1,tahun dan setelahnya wajib mendaftar kembali k...,None,None
2,"Kecil Min. 300 Juta h t s.d. 2,5 Milyar 6 dan ...",None,None
3,orang PJTBU dengan SKK minimal,None,None
4,per,None,None


## Create dataframe

### Subtask:
Organize the extracted information into a pandas DataFrame.


**Reasoning**:
Create a pandas DataFrame from the extracted lists and display the head.



In [19]:
import pandas as pd

df_extracted = pd.DataFrame({
    'NAMA PERUSAHAAN': company_names,
    'KUALIFIKASI': qualifications,
    'E-MAIL USAHA': emails
})

display(df_extracted.head())

,NAMA PERUSAHAAN,KUALIFIKASI,E-MAIL USAHA
0,Direktori Perusahaan Konstruksi Provinsi Jawa ...,None,None
1,tahun dan setelahnya wajib mendaftar kembali k...,None,None
2,"Kecil Min. 300 Juta h t s.d. 2,5 Milyar 6 dan ...",None,None
3,orang PJTBU dengan SKK minimal,None,None
4,per,None,None


## Process data

### Subtask:
Retry extracting company names, qualifications, and email addresses by iterating through the text lines and applying more robust regular expressions and pattern matching to identify and extract the required information.


**Reasoning**:
Retry extracting company names, qualifications, and email addresses by iterating through the text lines and applying more robust regular expressions and pattern matching to identify and extract the required information.



In [20]:
import re

company_names = []
qualifications = []
emails = []

# Regex patterns to identify company names, qualifications, and emails
# This is a basic pattern and might need adjustment based on the actual data format
company_pattern = re.compile(r'^\d+\s+(.*)') # Starts with a number, followed by company name
qualification_pattern = re.compile(r'\b(BESAR|MENENGAH|KECIL)\b') # Looks for specific qualification keywords
email_pattern = re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}') # Standard email pattern


current_company_name = None
current_qualification = None
current_email = None

for i, line in enumerate(all_lines):
    line = line.strip() # Remove leading/trailing whitespace

    # Check if the line starts a new company entry
    if re.match(r'^\d+\s+', line): # Line starts with a number followed by a space
        # If we were processing a previous company, append its data
        if current_company_name is not None or current_qualification is not None or current_email is not None:
            company_names.append(current_company_name)
            qualifications.append(current_qualification)
            emails.append(current_email)

        # Reset for the new company
        current_company_name = None
        current_qualification = None
        current_email = None

        # Try to extract information from the new line
        company_match = company_pattern.match(line)
        if company_match:
            current_company_name = company_match.group(1).strip()

        qualification_match = qualification_pattern.search(line)
        if qualification_match:
            current_qualification = qualification_match.group(1)

        email_match = email_pattern.search(line)
        if email_match:
            current_email = email_match.group(0)

    else:
        # If not a new company line, try to find qualification or email in the current line
        qualification_match = qualification_pattern.search(line)
        if qualification_match:
            current_qualification = qualification_match.group(1)

        email_match = email_pattern.search(line)
        if email_match:
            current_email = email_match.group(0)


# After the loop, append the last company's data
if current_company_name is not None or current_qualification is not None or current_email is not None:
    company_names.append(current_company_name)
    qualifications.append(current_qualification)
    emails.append(current_email)

## Create dataframe

### Subtask:
Organize the extracted information into a pandas DataFrame.


**Reasoning**:
Create a pandas DataFrame named `df_extracted` using the `company_names`, `qualifications`, and `emails` lists with the specified column names and display the head.



In [21]:
import pandas as pd

df_extracted = pd.DataFrame({
    'NAMA PERUSAHAAN': company_names,
    'KUALIFIKASI': qualifications,
    'E-MAIL USAHA': emails
})

display(df_extracted.head())

,NAMA PERUSAHAAN,E-MAIL USAHA
0,Direktori Perusahaan Konstruksi Provinsi Jawa ...,None
1,tahun dan setelahnya wajib mendaftar kembali k...,None
2,"Kecil Min. 300 Juta h t s.d. 2,5 Milyar 6 dan ...",None
3,orang PJTBU dengan SKK minimal,None
4,per,None


In [24]:
display(df_extracted.tail())

,NAMA PERUSAHAAN,E-MAIL USAHA
16796,TODAN PUTRA Tidak memiliki Lainnya JL LAYA/N/G...,priyonotedi@gmail.com
16797,"WIJAYA KUSUMA, CV Tidak memiliki CV pJL BRAWIJ...",griabrawijaya@yahoo.co.id
16798,Direktori Perusahaan Konstruksi Provinsi Jawa ...,None
16799,Direktori Perusahaan Konstruksi Provinsi Jawa ...,None
16800,Direktori Perusahaan Konstruksi Provinsi Jawa ...,None


## Save to csv

### Subtask:
Save the DataFrame to a CSV file.


In [22]:
#save to csv
df_extracted.to_csv('extracted_data.csv', index=False)
print("Data saved to extracted_data.csv")

#cleaned data

In [25]:
#Cleaned data
df_extracted = df_extracted.dropna(subset=['E-MAIL USAHA'])


In [26]:
df_extracted.describe()

,NAMA PERUSAHAAN,E-MAIL USAHA
count,6134,6134
unique,6130,6089
top,"SOLO DUNIA BAJA Kecil PT/PT KEBUMEN, KARANGDUW...",soloduniabaja@gmail.com
freq,5,6


In [27]:
#remove duplicate email
df_extracted = df_extracted.drop_duplicates(subset=['E-MAIL USAHA'])

In [28]:
df_extracted.describe()

,NAMA PERUSAHAAN,E-MAIL USAHA
count,6089,6089
unique,6089,6089
top,"WIJAYA KUSUMA, CV Tidak memiliki CV pJL BRAWIJ...",griabrawijaya@yahoo.co.id
freq,1,1


In [30]:
df_extracted.columns

Index(['NAMA PERUSAHAAN', 'E-MAIL USAHA'], dtype='object')

In [33]:
df_extracted['NAMA PERUSAHAAN'][40]

'CAHAYA AGUNG BAROKAH, PT Menengah PT/PT DSN SUREN, RT 03 RW 03, TApMBAKREJA, KEDUNGREJA - cahaya_caba47@yahoo.com'

In [36]:
df_extracted['E-MAIL USAHA'][40]

'cahaya_caba47@yahoo.com'

In [35]:
df_extracted['NAMA PERUSAHAAN'][60]

'FAJAR MAS MURNI, PT Menengah PT/PT JL SETIA BUDI, RT 05 RW 04, KEBONMgANIS, CILACAP Konstruksi Khusus legal@fajarmasmurni.com'

,NAMA PERUSAHAAN,E-MAIL USAHA,NAMA PERUSAHAAN_1,NAMA PERUSAHAAN_2,NAMA PERUSAHAAN_3,NAMA PERUSAHAAN_4,NAMA PERUSAHAAN_5,NAMA PERUSAHAAN_6,NAMA PERUSAHAAN_7
18,"FUDING MANDIRI SEJAHTERA, PT Besar PT/PT JL SE...",dwiansky@yahoo.co.id,FUDING MANDIRI SEJAHTERA,PT Besar PT/PT JL SENGON,RT 01 RW 04,TRITIH KULON,CILACAP UTARA - Konstruksi Sipil dwiansky@yaho...,None,None
19,"MELISTA KARYA, PT Besar PT/PT JL DI PANJAITAN ...",melista_karya@yahoo.com,MELISTA KARYA,PT Besar PT/PT JL DI PANJAITAN NO 47 A,RT 02 RW 06,DON.AN,CILACAP Konstruksi Sipil melista_karya@yahoo.com,None,None
31,"ARUM BINA CILACAP, PT Menengah PT/PT PERUM GUM...",arum.bina@gmail.com,ARUM BINA CILACAP,PT Menengah PT/PT PERUM GUMILIR INDAH JL CEMAR...,RT 03 RW Konstruksi Gedung arum.bina@gmail.com,None,None,None,None
39,"BUMI INTAN BERCAHAYA, PT Menengah PT/PT JL DR ...",bumiintanbercahaya@gmail.com,BUMI INTAN BERCAHAYA,PT Menengah PT/PT JL DR CIPTO NO 18,RT 03 RW 03,KEBgONMANIS,CILACAP Konstruksi Gedung bumiintanbercahaya@g...,None,None
40,"CAHAYA AGUNG BAROKAH, PT Menengah PT/PT DSN SU...",cahaya_caba47@yahoo.com,CAHAYA AGUNG BAROKAH,PT Menengah PT/PT DSN SUREN,RT 03 RW 03,TApMBAKREJA,KEDUNGREJA - cahaya_caba47@yahoo.com,None,None


In [38]:
# Select only the desired columns
df_extracted = df_extracted[['E-MAIL USAHA', 'NAMA PERUSAHAAN_1', 'NAMA PERUSAHAAN_5']]

# Display the updated DataFrame
display(df_extracted.head())

,E-MAIL USAHA,NAMA PERUSAHAAN_1,NAMA PERUSAHAAN_5
18,dwiansky@yahoo.co.id,FUDING MANDIRI SEJAHTERA,CILACAP UTARA - Konstruksi Sipil dwiansky@yaho...
19,melista_karya@yahoo.com,MELISTA KARYA,CILACAP Konstruksi Sipil melista_karya@yahoo.com
31,arum.bina@gmail.com,ARUM BINA CILACAP,None
39,bumiintanbercahaya@gmail.com,BUMI INTAN BERCAHAYA,CILACAP Konstruksi Gedung bumiintanbercahaya@g...
40,cahaya_caba47@yahoo.com,CAHAYA AGUNG BAROKAH,KEDUNGREJA - cahaya_caba47@yahoo.com


In [40]:
df_extracted['NAMA PERUSAHAAN_5'][18]

'CILACAP UTARA - Konstruksi Sipil dwiansky@yahoo.co.id'

In [41]:
# Remove email addresses from the end of the 'NAMA PERUSAHAAN_5' column
df_extracted['NAMA PERUSAHAAN_5'] = df_extracted['NAMA PERUSAHAAN_5'].str.replace(r'\s*[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', '', regex=True)

# Display the updated DataFrame
display(df_extracted.head())

,E-MAIL USAHA,NAMA PERUSAHAAN_1,NAMA PERUSAHAAN_5
18,dwiansky@yahoo.co.id,FUDING MANDIRI SEJAHTERA,CILACAP UTARA - Konstruksi Sipil
19,melista_karya@yahoo.com,MELISTA KARYA,CILACAP Konstruksi Sipil
31,arum.bina@gmail.com,ARUM BINA CILACAP,None
39,bumiintanbercahaya@gmail.com,BUMI INTAN BERCAHAYA,CILACAP Konstruksi Gedung
40,cahaya_caba47@yahoo.com,CAHAYA AGUNG BAROKAH,KEDUNGREJA -


In [42]:
# Rename the columns
df_extracted = df_extracted.rename(columns={
    'NAMA PERUSAHAAN_1': 'NAMA PERUSAHAAN',
    'NAMA PERUSAHAAN_5': 'ALAMAT DAN JENIS'
})

# Display the updated DataFrame with new column names
display(df_extracted.head())

,E-MAIL USAHA,NAMA PERUSAHAAN,ALAMAT DAN JENIS
18,dwiansky@yahoo.co.id,FUDING MANDIRI SEJAHTERA,CILACAP UTARA - Konstruksi Sipil
19,melista_karya@yahoo.com,MELISTA KARYA,CILACAP Konstruksi Sipil
31,arum.bina@gmail.com,ARUM BINA CILACAP,None
39,bumiintanbercahaya@gmail.com,BUMI INTAN BERCAHAYA,CILACAP Konstruksi Gedung
40,cahaya_caba47@yahoo.com,CAHAYA AGUNG BAROKAH,KEDUNGREJA -


In [44]:
# Reorder the columns
df_extracted = df_extracted[['NAMA PERUSAHAAN', 'E-MAIL USAHA', 'ALAMAT DAN JENIS']]

# Display the updated DataFrame with reordered columns
display(df_extracted.head())

,NAMA PERUSAHAAN,E-MAIL USAHA,ALAMAT DAN JENIS
18,FUDING MANDIRI SEJAHTERA,dwiansky@yahoo.co.id,CILACAP UTARA - Konstruksi Sipil
19,MELISTA KARYA,melista_karya@yahoo.com,CILACAP Konstruksi Sipil
31,ARUM BINA CILACAP,arum.bina@gmail.com,None
39,BUMI INTAN BERCAHAYA,bumiintanbercahaya@gmail.com,CILACAP Konstruksi Gedung
40,CAHAYA AGUNG BAROKAH,cahaya_caba47@yahoo.com,KEDUNGREJA -


In [45]:
#SAVE TO CSV
df_extracted.to_csv('extracted_data.csv', index=False)
print("Data saved to extracted_data.csv")

Data saved to extracted_data.csv
